# Reflection and Blogpost Writing

Here we will learn about **reflection agentic design pattern**, through a simple example of writing a blog post.

---

## Pre-requisites

- Create conda env using the following command from the root of this repo: `conda env create -f ./notebooks/multi-agent-systems/pyautogen/conda-env.yaml`.
- Please define the following environment variables in `.env` file:
    - `OPENAI_API_BASE="<ENDPOINT_URL>"`
    - `OPENAI_API_KEY="<API_KEY>"`

## Introduction

- We will use **nested chat conversation pattern** to realize a sophisticated reflection process.
- We will build a system where a set of reviewer agents is nested within a critic agent, to reflect on a blog post written by a writer agent.

## Setup

In [22]:
import warnings
import os
from dotenv import load_dotenv, find_dotenv

warnings.filterwarnings('ignore')
_ = load_dotenv(find_dotenv())

Please find additional info about how to create LLM Configuration [over here](https://microsoft.github.io/autogen/docs/topics/llm_configuration).

In [23]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_type": "azure",
    "base_url": os.environ.get("OPENAI_API_BASE"),
    "api_key": os.environ.get("OPENAI_API_KEY"),
}

## The task

In [24]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

## Create a writer agent

In [25]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [26]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

### Unveiling the Mysteries of AI with DeepLearning.AI

In the rapidly evolving arena of artificial intelligence, DeepLearning.AI stands out as a beacon for those looking to dive into the complexities of AI and machine learning. Founded by Andrew Ng, a pioneer in the AI industry, this educational platform offers a plethora of courses designed to equip novices and professionals alike with the skills necessary in today’s tech-driven world. Through their in-depth tutorials and hands-on projects, learners understand AI algorithms and apply them practically. Whether you're starting from scratch or looking to polish your skills, DeepLearning.AI guides you through each step of mastering this cutting-edge technology.


## Adding reflection

### Two-agents reflection

Create a critic agent to reflect on the work of the writer agent.

<img src="./images/two-agents-reflection.png" alt="Customer Onboarding Overview" style="width: 30%;"/>

In [27]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [28]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Revolutionizing AI Education: A Closer Look at DeepLearning.AI

DeepLearning.AI is an innovative education technology company, spearheaded by AI pioneer Andrew Ng. Since its inception, it has become a beacon for professionals and enthusiasts alike, eager to harness the power of artificial intelligence. The platform offers a plethora of courses from AI fundamentals to cutting-edge specializations. What sets DeepLearning.AI apart is its community-focused approach, providing learners with valuable networking opportunities and real-world project experience. Whether you’re a novice or a seasoned expert, DeepLearning.AI equips you with the tools necessary to thrive in the era of AI. Ready to future-proof your career? Dive into DeepL

### Nested chat reflection

<img src="./images/nested-chat-reflection.png" alt="Nested Chat Reflection" style="width: 40%;"/>

In [29]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [30]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [31]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [32]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

#### Orchestrate the nested chats to solve the task

In [33]:
def reflection_message(recipient, messages, sender, config):
    return f"""Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"""


review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'reviewer': '', 'review': ''}",
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1,
    },
]

In [34]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,  # wnenever writer sends a message, the review_chats will be triggered
)

In [35]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Revolutionizing AI Education: A Closer Look at DeepLearning.AI

DeepLearning.AI is an innovative education technology company, spearheaded by AI pioneer Andrew Ng. Since its inception, it has become a beacon for professionals and enthusiasts alike, eager to harness the power of artificial intelligence. The platform offers a plethora of courses from AI fundamentals to cutting-edge specializations. What sets DeepLearning.AI apart is its community-focused approach, providing learners with valuable networking opportunities and real-world project experience. Whether you’re a novice or a seasoned expert, DeepLearning.AI equips you with the tools necessary to thrive in the era of AI. Ready to future-proof your career? Dive into DeepL

#### Get the summary

In [37]:
print(res.summary)

### Revolutionizing AI Education: A Closer Look at DeepLearning.AI

DeepLearning.AI is transcending borders in AI education through its comprehensive online platform. Founded by AI luminary Andrew Ng, it offers a spectrum of AI courses, from beginner basics to advanced techniques. This AI education platform doesn't just educate; it integrates community building, helping learners network and collaborate on real-world projects. Whether you're looking to start your journey in artificial intelligence or polish your skills, DeepLearning.AI provides the tools needed for success in today's tech-driven world. Embrace the future of learning; explore how to learn artificial intelligence through DeepLearning.AI today.

Meta Description: Discover DeepLearning.AI, your ultimate AI learning platform with courses designed for all levels. Join a global community and enhance your career in AI.
